In [2]:
cd started_from_the_bottom

c:\Users\user\Desktop\workbench\started_from_the_bottom


In [3]:
import pandas as pd 
from kiwipiepy import Kiwi
from FINAL.module.jh import JH
import numpy as np
import re

In [4]:
kiwi = Kiwi(model_type="sbg", num_workers=0)

In [334]:
df = pd.read_excel(r"C:\Users\user\Desktop\workbench\dataset\offerTrain.xlsx")

In [6]:
sido = pd.read_csv(r"C:\Users\user\Desktop\workbench\dataset\국토교통부_전국 법정동_20221031.csv", encoding= 'cp949')

In [50]:
world = pd.read_csv(r"C:\Users\user\Desktop\workbench\dataset\외교부_국가(지역)별 일반현황_20220629.csv", encoding= 'cp949')

In [97]:
company = pd.read_csv(r"C:\Users\user\Desktop\workbench\dataset\total_Info.csv")

In [107]:
company_list = company["stock_name"].unique().tolist()

In [108]:
len(company_list)

90459

In [448]:
sido = sido[["시도명","읍면동명"]]

In [9]:
df[df["표준상품명"] == "-"]

,사업자등록번호,상점명,카테고리명,상품명,표준상품명,가격
0,1464300910,BGT단팥빵_불광점,전체메뉴,-3000,-,-3000
1,1464300910,BGT단팥빵_불광점,전체메뉴,-2000,-,-2000
2,2200284777,미각상점 강남역지하상가점,마카롱/뚱카롱,0,-,0
3,2200284777,미각상점 강남역지하상가점,마카롱/뚱카롱,0,-,0
4,2200284777,미각상점 강남역지하상가점,마카롱/뚱카롱,0,-,0
...,...,...,...,...,...,...
310254,3842501045,구구족_이대점,반반세트,족발보쌈 대\n(구구족 1족+보쌈 소),-,36900
310255,1931401986,구구족_대치점,반반세트,족발보쌈 대\n(구구족 1족+보쌈 소),-,36900
310256,6560302758,일만족발,내맘대로 반반 세트,족발(2인)+불족(2인),-,34800
310257,6560302758,일만족발,내맘대로 반반 세트,족발(2인)+불족(300g),-,28800


In [215]:
stop_words = ["정식","안주"]

In [75]:
text = "달콤한아이스초코라떼"

#### 불용어 알고리즘

In [217]:
def is_stop_words(text, stop_words) :
    result = kiwi.space(text).split(" ")
    for i in result :
        if i in stop_words :
            pat = re.compile(f"{i}")
            text = re.sub(pat, "", text)
    return text.strip()


In [213]:
func = lambda txt : is_stop_words(txt, stop_words)

In [218]:
func("돼지 고기 김치찌개 정식")

'돼지 고기 김치찌개'

#### 옵션 알고리즘

In [9]:
def is_option(text, a) :
    result = kiwi.space(text).split(" ")
    for i in result :
        if i in a :
            pat = re.compile(f"{i}")
            valiable = "".join(pat.findall(text))
            return " ".join(result), f"옵션 {valiable}"


In [109]:
option_list = ["추가","선택", "많이", "적게", "조금","빼고", "리필"]

In [10]:
option_func = lambda txt : is_option(txt, option_list)

In [88]:
kiwi.space("아보카도추가")

'아보카도 추가'

In [107]:
option_func("물조금")

('물 조금', '옵션 조금')

### 특수문자 algorithm

특수문자가 오는 경우 단어 분리(명사인 경우만)

제거

등심 돈가스 + 아이스크림

input1
돈가스 / 등심 

input2
아이스크림 




In [178]:
test_word = "라면+아이스크림"

In [116]:
test_word.split("+")

['과자 ', ' 아이스크림']

In [232]:
split_result = re.split(r"\+|\&|\/", test_word) # 단어 분리

In [233]:
for i in split_result :
    print(split_result[split_result.index(i)].strip())

*오픈기념* 1
1 아이스크림


In [234]:
c = [split_result[split_result.index(i)].strip() for i in split_result]

In [268]:
print(next(kiwi.tokenize(c)))

[Token(form='*', tag='SW', start=0, len=1), Token(form='오픈', tag='NNG', start=1, len=2), Token(form='기념', tag='NNG', start=3, len=2), Token(form='*', tag='SW', start=5, len=1), Token(form='1', tag='SN', start=7, len=1)]


#### 특수문자 알고리즘

In [11]:
def special_char(text) : 
    split_result = re.split(r"\+|\&|\/|\,", text)
    if len(split_result) >= 2 :
        combine_text = [split_result[split_result.index(i)].strip() for i in split_result]
        pattern = re.compile(r"[^\w\s]+")
        return [pattern.sub("", i ) for i in combine_text]
        # text_token = [j for i in kiwi.tokenize(combine_text) for j in i]
        # cp = tuple(cp_data.form for cp_data in text_token if cp_data.tag.startswith("N"))
        # return cp
    else : 
        pattern = re.compile(r"[^\w\s]+")
        return kiwi.space(pattern.sub("", text))


In [279]:
text_token = [j for i in kiwi.tokenize(c) for j in i]

In [380]:
test_word = "*오픈기념* 1+1 아메리카노 & 치킨브레스트"

In [385]:
pat_result = re.split(r"\+|\&|\/", test_word)

In [386]:
pat_result

['*오픈기념* 1', '1 아메리카노 ', ' 치킨브레스트']

In [395]:
special_char(test_word)

['오픈기념 1', '1 아메리카노', '치킨브레스트']

In [404]:
for i in kiwi.tokenize("강릉시 오뚜기 비빔밥") :
    print(i.form)

강릉시
오뚜기
비빔밥


In [141]:
company[company["stock_name"].str.contains("오뚜기")]

,corp_code,status,message,corp_name,corp_name_eng,stock_name,stock_code,ceo_nm,corp_cls,jurir_no,bizr_no,adres,hm_url,ir_url,phn_no,fax_no,induty_code,est_dt,acc_mt
2141,141529,0,정상,(주)오뚜기,OTTOGI CORPORATION,오뚜기,7310.0,"함영준, 황성만",Y,1341110002258.0,1388103238.0,경기도 안양시 동안구 흥안대로 405,www.ottogi.co.kr,NaN,031-421-2122,02) 528 -1970,10749.0,19710603.0,12.0
3545,170390,0,정상,오뚜기라면지주 주식회사,OTTOGI RAMYON CORPORATION,오뚜기라면지주,NaN,백창규,E,1347110003212,1258105093,"경기도 평택시 안중읍 덕우로 423 (용성리, 오뚜기라면)",NaN,NaN,031-683-1814,031-683-1242,10,19871130.0,12.0
3992,181095,0,정상,오뚜기냉동식품주식회사,"ottogi frozen foods co., Ltd.",오뚜기냉동식품,NaN,이 춘 원,E,1156110002780,1288104773,경기도 파주시 파주읍 파발로 65,NaN,NaN,031-954-7300,031-954-7304,10712,19870401.0,12.0
4224,187071,0,정상,오뚜기제유지주주식회사,"OTTOGI SESAME MILLS CO.,LTD",오뚜기제유지주,NaN,정병상,E,1345110002838.0,3038112229,충청북도 음성군 생극면 음성로 1506-27 (병암리),없음,NaN,043-877-4461,043-877-4464,10402,19801227.0,12.0
7375,255734,0,정상,오뚜기물류서비스지주주식회사,OTTOGILOGISTICS SERVICE HOLDINGS,오뚜기물류서비스지주,NaN,한태준,E,1345110017142.0,1358116883,"경기도 용인시 처인구 백암면 백암로 49 (백봉리, 오뚜기물류서비스)",www.ols.co.kr,NaN,031-284-4064,031-283-2862,52.0,19951005.0,12.0
9646,310217,0,정상,오뚜기에스에프지주 주식회사,"OTTOGISF HOLDINGS CO.,LTD",오뚜기에스에프지주,NaN,도영현,E,1950110002447.0,6128108783,경상남도 고성군 고성읍 남해안대로 2771,없음,NaN,055-672-9741,055-672-4748,64992,19981120.0,12.0
15151,420073,0,정상,오뚜기삼화식품(주),"OTTOGI SAMHWA FOODS CO.,LTD",오뚜기삼화식품,NaN,황종호,E,1350110040358,1348118535,충청남도 논산시 은진면 관촉로58번길 138 (성평리),NaN,NaN,041-730-9000,041-734-9008,10792,19931117.0,12.0
65244,1264924,0,정상,오뚜기에스에프 주식회사,OTTOGISF CORPORATION,오뚜기에스에프,NaN,도영현,E,1950110020952.0,3478700822,경상남도 고성군 고성읍 남해안대로 2771 오뚜기에스에프 주식회사(대평리),NaN,NaN,055-672-9741,055-672-4748,10211.0,20170801.0,12.0
76653,1430837,0,정상,오뚜기제유주식회사,"OTTOGI SESAME MILLS CO.,LTD",오뚜기제유,NaN,정병상,E,1545110085187.0,4698601269,충청북도 음성군 생극면 음성로 1506-27 (병암리),NaN,NaN,0438774461,0438774464,10402.0,20190801.0,12.0
83337,1523863,0,정상,오뚜기물류서비스(주),OTTOGILOGISTICS SERVICE,오뚜기물류서비스,NaN,한태준,E,1345110487949.0,6038802071,"경기도 용인시 기흥구 중부대로874번길 3 (상하동, 오뚜기물류서비스수원센타)",ols.co.kr/,NaN,031-283-2861,031-283-2862,52992.0,20201202.0,12.0


#### 고유명사 알고리즘

In [210]:
def nnp_extract(text, nnp_list) :
    res = []
    # split_text = kiwi.space(text) 
    for i in text.split() :  
    # for i in split_text.split() :    
        if i not in nnp_list :
            res.append(i)
    return " ".join(res)
                     

In [211]:
abc = lambda x: nnp_extract(x, sido)

In [207]:
abc("강릉 오뚜기 비빔밥")

강릉 오뚜기 비빔밥


'비빔밥'

In [187]:
df["상품명"].sample(500).apply(abc)

In [196]:
sido[sido.index("이탈리아")]

'이탈리아'

In [209]:
df[df["상품명"].str.contains("이탈리아")]["상품명"].apply(abc)

이탈리아노
이탈리아 노 프레즐 세트
이탈리아 노 프레즐 세트
요헤 (이탈리아)
(이탈리아 탄산음료) 산펠레그리노 과일 탄산음료
(이탈리아 탄산수) 산펠레그리노
본샹스 모스카토 골든 에디션 이탈리아
본샹스 모스카토 골든 에디션(이탈리아)
62. 요거트(이탈리아)
이탈리아 노 티라미수
이탈리아 노 티라미수
이탈리아 노 티라미수 케익
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크
이탈리아 노 티라미스 케이크

35404                          이탈리아노
69990                       노 프레즐 세트
70021                       노 프레즐 세트
145839                     요헤 (이탈리아)
221231    (이탈리아 탄산음료) 산펠레그리노 과일 탄산음료
                     ...            
395809                        노 티라미스
395811                        노 티라미스
395812                        노 티라미스
395814                        노 티라미스
395816                        노 티라미스
Name: 상품명, Length: 64, dtype: object

In [176]:
nnp_extract("강릉 오뚜기 비빔밥", sido)

'비빔밥'

In [436]:
nnp_set = set(i[0] for i in test if i)

In [82]:
test1 = sido["시군구명"].str[:-1].unique().tolist()

In [83]:
test1

['삼척',
 '춘성',
 '명주',
 '안성',
 '남양주',
 '용인',
 '연천',
 '고양',
 '광주',
 '울진',
 '시흥',
 '파주',
 '춘천',
 '화성',
 '김포',
 '여주',
 '포천',
 '평택',
 '양평',
 '원주',
 '홍천',
 '강화',
 '충주',
 '횡성',
 '이천',
 '옹진',
 '정선',
 '평창',
 '원성',
 '강릉',
 '철원',
 '양구',
 '가평',
 '천안',
 '양양',
 '영월',
 '고성',
 '청주시상당',
 '청주시서원',
 '청주시흥덕',
 '제천',
 '청주',
 '동해',
 '태백',
 '속초',
 '화천',
 '양주',
 '인제',
 '청주시청원',
 '서천',
 '보령',
 '청원',
 '홍성',
 '공주',
 '청양',
 '연기',
 '예산',
 '보은',
 '서산',
 '아산',
 '논산',
 '당진',
 '부여',
 '태안',
 '제원',
 '대전',
 '단양',
 '중원',
 '대전시동',
 '대전시중',
 '금산',
 '대전시서',
 '대덕',
 '영동',
 '증평',
 '진천',
 '옥천',
 '음성',
 '괴산',
 '천원',
 '계룡',
 '천안시동남',
 '천안시서북',
 '온양',
 '대천',
 '강서',
 '연제',
 '수영',
 '사상',
 '기장',
 '수원',
 '수원시장안',
 '해운대',
 '중',
 '동두천',
 '안산',
 '부천시남',
 '남',
 '북',
 '용산',
 '부평',
 '계양',
 '서',
 '남동',
 '유성',
 '인천',
 '동',
 '오산',
 '의정부',
 '금정',
 '성동',
 '인천시중',
 '성남',
 '인천시남',
 '용인시처인',
 '성북',
 '강북',
 '도봉',
 '노원',
 '광진',
 '동대문',
 '송탄',
 '울주',
 '군포',
 '의왕',
 '하남',
 '구로',
 '동작',
 '미금',
 '수원시권선',
 '수원시팔달',
 '수원시영통',
 '수성',
 '달서',
 '광산',
 '동래',
 '안

In [73]:
test2 = sido["시도명"].str[:2].unique().tolist()

In [91]:
sido_list = test1 + test2 + test4

In [47]:
sido_list = sido_list + test3

In [89]:
test4 = world["한글 국가명"].tolist()

In [111]:
sido = sido_list + company_list

In [114]:
sido = list(set(sido))

In [219]:
import pickle

In [225]:
with open("nnp_list.pkl", "wb") as f :
    pickle.dump(sido, f)

In [226]:
with open("nnp_list.pkl", "rb") as r :
    nnp_list = pickle.load(r)

In [231]:
with open(r"WD.WordToStandard.p", "rb") as r :
    word_dict = pickle.load(r)

In [233]:
len(word_dict.keys())

7683

#### 단어 표준화 알고리즘

In [241]:
for k, v in word_dict.items() :
    if k == v :
        print(k)

-
라면사리
오뚜기
사리면
가지볶음
건가지볶음
강하개
고구마
큐브
건강하개
고구마스틱
강한
우리집
잡채
건강한
과일
건과일
초코
망디앙
두부
건두부
두부무침
건두부무침
두부볶음
건두부볶음
레몬
건레몬
망고
건망고
면
건면
무화과
건무화과
미역
건미역
빙어
건빙어
새우
건새우
달래전
마늘쫑볶음
야채전
새우볶음
건새우볶음
생선
건생선
식
용지
건식
어물
건어물
어물소스
건어물소스
오징어
건오징어
자두
다쿠아즈
건자두
전지
건전지
채소
건채소
취나물
건취나물
크랜베리
건크랜베리
포도
스콘
건포도
아몬드
안주
치즈
브레드
호두
감귤
주스
생감귤
라떼
생고구마
고기김치찌개
생고기김치찌개
도시락
생과일
빙수
빵
와플
타르트
화채
굴
생굴
짬봉밥
짬뽕
짬뽕밥
귤
에이드
생귤
귤청
생귤청
김치
밀키트
생김치
깻잎김치
생깻잎김치
깻잎무침
생깻잎무침
깻잎절임
생깻잎절임
꼼장어
생꼼장어
달걀
생달걀
규동
닭
생닭
닭고기스틱
생닭고기스틱
대구탕
생대구탕
대패삼겹살
생대패삼겹살
더덕무침
생더덕무침
돈가스
생돈가스
생두부
등심
생등심
딸기
생딸기
그릭요거트
밀크
밀크티
버블티
산도
생크림
카스텔라
크루아상와플
슈크림
스무디
시리얼
파르페
연유
요거트
샤베트
우유
케이크
크루아상
크림
도넛
바움쿠헨
크림치즈
탄산수
티라미수
딸기차
생딸기차
딸기청
생딸기청
딸기파르페
생딸기파르페
똥집
양념
생똥집
라임
스파클링
생라임
워터
블랙티
생레몬
생강차
오렌지
레몬차
생레몬차
마늘
생마늘
막걸리
생막걸리
맛고기
생맛고기
생망고
멜론
생멜론
생면
명란
아보카도
샐러드
생명란
바나나
생바나나
복숭아
키위
바질
파스타
생바질
생복숭아
블루베리
생블루베리
사과
당근
생사과
생새우
야끼교자
초밥
수박
생수박
생아보카도
샌드위치
양파
생양파
연어
생연어
덮밥
롤
라이스볼
포케
샐러드볼
아보카도볼
유부
카르파쵸
포케볼
연어동
생연어동
구운연어
연어회
생연어회
오디
생오디
생오렌지
오리
주물럭
야채볶음
생오리
오이
피클
생오이
와사비
생와사비
게살
삼겹살
김밥
소고기
참치
유자
셰이크
생유자
자몽
생자몽
자몽차

In [266]:
def word_standard(txt) :
    res = []
    for word in txt.split(' '):
        try:
            res.append(word_dict[word])
        except :
            res.append(word)
    return " ".join(res)
        

In [251]:
word_standard("마파두부 김치찌개 사발국 사발라면 마라탕 ")

우선 시도 마파두부
우선 시도 김치찌개
우선 시도 사발국
실패 사발국
우선 시도 사발라면
실패 사발라면
우선 시도 마라탕


'마파두부 김치찌개 사발국 사발라면 마라탕'

In [252]:
with open("WD.StandardDict.p", "rb") as r :
    StandradDict = pickle.load(r)

In [260]:
for k, v in StandradDict.items() :
    try : 
        print(v["wordList"])    
    except :
        print("#### 실패 ####", k)
    

{'-'}
{'라면사리'}
{'오뚜기'}
{'사리면'}
{'가지볶음'}
{'건가지볶음'}
{'강하개'}
{'고구마'}
{'큐브'}
{'건강하개'}
{'고구마스틱'}
{'강한'}
{'우리집'}
{'잡채'}
{'건강한'}
{'과일'}
{'건과일'}
{'초코'}
{'망디앙'}
{'두부'}
{'건두부'}
{'두부무침'}
{'건두부무침'}
{'두부볶음'}
{'건두부볶음'}
{'레몬'}
{'건레몬'}
{'망고'}
{'건망고'}
{'면'}
{'건면'}
{'무화과'}
{'건무화과'}
{'미역'}
{'건미역'}
{'빙어'}
{'건빙어'}
{'새우'}
{'건새우'}
{'달래전'}
{'마늘쫑볶음'}
{'야채전'}
{'새우볶음'}
{'건새우볶음'}
{'생선'}
{'건생선'}
{'식'}
{'용지'}
{'건식'}
{'어물'}
{'건어물'}
{'어물소스'}
{'건어물소스'}
{'오징어'}
{'건오징어'}
{'자두'}
{'다쿠아즈'}
{'건자두'}
{'전지'}
{'건전지'}
{'채소'}
{'건채소'}
{'취나물'}
{'건취나물'}
{'크랜베리'}
{'건크랜베리'}
{'포도'}
{'스콘'}
{'건포도'}
{'아몬드'}
{'안주'}
{'치즈'}
{'브레드'}
{'호두'}
{'감귤'}
{'주스'}
{'생감귤'}
{'라떼'}
{'생고구마'}
{'고기김치찌개'}
{'생고기김치찌개'}
{'도시락'}
{'생과일'}
{'빙수'}
{'빵'}
{'와플'}
{'타르트'}
{'화채'}
{'굴'}
{'생굴'}
{'짬봉밥'}
{'짬뽕'}
{'짬뽕밥'}
{'귤'}
{'에이드'}
{'생귤'}
{'귤청'}
{'생귤청'}
{'김치'}
{'밀키트'}
{'생김치'}
{'깻잎김치'}
{'생깻잎김치'}
{'깻잎무침'}
{'생깻잎무침'}
{'깻잎절임'}
{'생깻잎절임'}
{'꼼장어'}
{'생꼼장어'}
{'달걀'}
{'생달걀'}
{'규동'}
{'닭'}
{'생닭'}
{'닭고기스틱'}
{'생닭고기스틱'}
{'대구탕'}
{'생대구탕'}
{'대패삼겹살'}
{'생대패삼겹살'}
{'더덕무침'}
{'생더덕무침'}
{'돈가스'}
{'생돈가스'}

In [265]:
for k, v in StandradDict.items() :
    if k == "초콜릿" :
        print(k)
        print(v)

초콜릿
{'wordList': {'초콜릿'}, 'jamo': 'ㅊㅗㅋㅗㄹㄹㅣㅅ', 'idxList': [1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 3, 2, 2, 2, 1, 3, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 3, 2, 2, 1, 3, 2, 2, 3, 2, 3, 2, 2, 2, 5, 2, 3, 3, 2, 2, 2, 2, 2, 3, 2, 3, 3, 2, 3, 3, 2, 2, 2, 2, 3, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 3, 3, 2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 3, 3, 2, 3, 3, 3, 2, 2, 3, 2, 2, 2, 2, 3, 2, 3, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 2], 'mean': 1.9401197604790419}


In [272]:
kiwi.analyze("1L")

[([Token(form='1', tag='SN', start=0, len=1),
   Token(form='L', tag='SL', start=1, len=1)],
  -15.325386047363281)]

In [275]:
kiwi.analyze("돼지두부김치찌개안주大")

[([Token(form='돼지', tag='NNG', start=0, len=2),
   Token(form='두부', tag='NNG', start=2, len=2),
   Token(form='김치찌개', tag='NNG', start=4, len=4),
   Token(form='안주', tag='NNG', start=8, len=2),
   Token(form='大', tag='SH', start=10, len=1)],
  -61.30607986450195)]

In [ ]:
def get_term_frequency(document, word_dict=None):
    if word_dict is None:
        word_dict = {}
    words = document.split()

    for w in words:
        word_dict[w] = 1 + (0 if word_dict.get(w) is None else word_dict[w])

    return pd.Series(word_dict).sort_values(ascending=False)

get_term_frequency(doc1)

In [276]:
df = pd.read_excel(r"C:\Users\user\Desktop\workbench\dataset\offerTrain.xlsx")

In [335]:
df = df[["상품명", "표준상품명"]]

In [336]:
df = df.drop_duplicates()

In [340]:
df

,상품명,표준상품명,상품명_길이,표준상품명_길이
0,-3000,-,5,1
1,-2000,-,5,1
2,0,-,1,1
10,2%,-,2,1
11,1,-,1,1
...,...,...,...,...
428671,ZEAL 뉴질랜드 해산물 리조또 3kg,ZEAL 뉴질랜드 해산물 리조또,21,17
428677,(샘플)zeal사료양고기리조또50g,ZEAL 사료 양고기 리조또,19,15
428678,zeal 사슴힘줄125g,ZEAL 사슴힘줄,13,9
428679,zeal 쇠고기져키125g,ZEAL 쇠고기져키,14,10


In [339]:
df["표준상품명_길이"] = df["표준상품명"].apply(lambda x : len(x))

In [286]:
df["길이차이"] = abs(df["표준길이"] - df["상품길이"])

In [343]:
df["길이_차이"] = df["표준상품명_길이"] - df["상품명_길이"]

In [300]:
len(df[df["길이차이"] == 1]["표준상품명"].unique())

9556

In [305]:
df[df["차이"] == 1][["상품명","표준상품명"]].drop_duplicates().sample(20)

,상품명,표준상품명
201905,베이컨 에그롤,베이컨 에그 롤
313599,차돌김밥,차돌 김밥
230080,생크림크루와상,생크림 크루아상
341761,치즈후렌치후라이,치즈 프렌치프라이
81406,감자라면,감자 라면
213930,블루베리요거트 케이크,블루베리 요거트 케이크
111783,계란김밥,달걀 김밥
410894,해물순두부찌개,해물 순두부찌개
79703,갈릭식빵,갈릭 식빵
384325,클라우드 무알콜맥주,클라우드 무알콜 맥주


In [306]:
k = df[df["차이"] == 1][["상품명","표준상품명"]].drop_duplicates()

In [319]:
def lam(row):
    if row['표준상품명'].replace(' ','') == row['상품명']:
        return True
    return False

In [323]:
# 요소요소 요소 요소
# k[k.apply(lam, axis=1) == True].sample(25)
# 표준화 변경
k[k.apply(lam, axis=1) == False].sample(25 )

,상품명,표준상품명
410760,씨푸드로제파스타,해물 로제 파스타
121639,돈까스돈부리,돈가스 돈부리
315307,참치마요 김밥 도시락,참치 마요 김밥 도시락
144842,라즈베리뚱카롱,라즈베리 마카롱
405727,피치망고 블라썸,피치 망고 블라썸
97406,*김맛 로우쏭케익,김맛 로우쏭 케이크
362302,캬라멜다쿠아즈,캐러멜 다쿠아즈
193973,발로나초코 마카롱,발로나 초코 마카롱
367372,커피 티라떼,커피 티 라떼
306138,잠봉뵈르 소금빵,잠봉뵈르 소금 빵


In [ ]:
# 요소요소 요소 요소
# k[k.apply(lam, axis=1) == True].sample(25)
# 표준화 변경
k[k.apply(lam, axis=1) == False].sample(25 )

In [332]:
df[df["길이차이"] >= 7 ][["상품명","표준상품명"]].sample(25)

,상품명,표준상품명
139969,[88핫도그] 땅콩독(자체),땅콩 핫도그
22603,미스터에그 세트,-
59902,유린기 소 + 칠리새우6,-
60383,(루프set)더블치즈 버거,-
23413,반했드랍 세트(배달용),-
29667,스캔들버거SET (떡볶이),-
294707,순수우유 쿠키슈EVENT,우유 쿠키슈
48124,햄 앤 치즈 길거리 토스트 세트(배달용),-
61878,딥치즈버거 세트,-
259221,리얼 아보카도 콥 샐러드(배달용),아보카도 샐러드


In [283]:
df[abs(df["표준길이"] - df["상품길이"]) <= 5][["표준상품명","표준길이"]]

,표준상품명,표준길이
0,-,1
1,-,1
2,-,1
3,-,1
4,-,1
...,...,...
428676,ZEAL 뉴질랜드 해산물 리조또,17
428677,ZEAL 사료 양고기 리조또,15
428678,ZEAL 사슴힘줄,9
428679,ZEAL 쇠고기져키,10


In [281]:
df["상품길이"]

0          5
1          5
2          1
3          1
4          1
          ..
428676    21
428677    19
428678    13
428679    14
428680    11
Name: 상품길이, Length: 428681, dtype: int64